In [1]:
from fabric.api import local
import json

In [2]:
# ex: fab github_check
def GithubCheck():
    local('curl -H "Authorization: token {0}" {1}/user'.format(oauth_token, github_api_url))

# ex: fab github_issue_comments:owner=boyd-mask,repo=git-jenkins-test,issue_num=2
def GithubListComments(owner, repo, issue_num):
    local('curl -H "Authorization: token {0}" {1}/repos/{2}/{3}/issues/{4}/comments'.format(oauth_token, github_api_url, owner, repo, issue_num))
    
# post comment to github
def GithubCreateComment(owner, repo, path_log, issue_num="1"):
    # read upper stream build log
    text = ReadFile(path_log)

    # extract issue number from build log
    issue_num = text.split(" ")[3].replace("#", "")
    
    # extract commit id from build log
    commit_id = text.split(" ")[6]
    
    # extract commit message
    commit_message = CommitMessage(owner, repo, commit_id)
    
    # dummy message
    greeting = "Hello Guys. I am Mr.Robot. \n"

    # json text
    comment = {'body': greeting + "```\n" + text + "```\n"}

    # posting comment
    local('curl -H \"Authorization: token {0}\" {1}/repos/{2}/{3}/issues/{4}/comments -X POST -d \' {5}\''.format(oauth_token, github_api_url, owner, repo, issue_num, json.dumps(comment)))

In [55]:
from fabric.api import local
import json

# Return str
def ReadFile(path):
  with open(path, 'r') as f:
    output = f.read().replace("'","\"")
  return output

# Return specific commit message
def CommitMessage(oauth_token, github_api_url, owner, repo, commit_id):
    commit_raw = local('curl -H "Authorization: token {0}" {1}/repos/{2}/{3}/commits/{4}'.format(oauth_token, github_api_url, owner, repo, commit_id), capture=True)
    message = json.loads(str(commit_raw))["commit"]["message"]
    return message

# Return last commit's message
def CurrentCommitMessage(oauth_token, github_api_url, owner, repo, git_current_branch, git_current_commit_id):
    commit_raw = local('curl -H "Authorization: token {0}" {1}/repos/{2}/{3}/commits/{4}'.format(oauth_token, github_api_url, owner, repo, git_current_commit_id), capture=True)
    git_head_commit_id = json.loads(str(commit_raw))['parents'][-1]['sha']
    print "git_head_commit_id: " + git_head_commit_id
    return CommitMessage(owner, repo, git_head_commit_id)

# post comment to github
def GithubCreateComment(oauth_token, github_api_url, owner, repo, git_current_branch, path_log):

    # extract issue number. branche names like : "origin/pr/3/merge"
    issue_num = git_current_branch.split('/')[2]
    
    # read upper stream build log
    text = ReadFile(path_log)
    
    # dummy message
    greeting = "Hello Guys. I am Mr.Robot. \n"

    # json text
    comment = {'body': greeting + "```\n" + text + "```\n"}

    # posting comment
    local('curl -H \"Authorization: token {0}\" {1}/repos/{2}/{3}/issues/{4}/comments -X POST -d \' {5}\''.format(oauth_token, github_api_url, owner, repo, issue_num, json.dumps(comment)))

In [ ]:
# default variables
oauth_token="6900343dac01229d1c616b1e6a5b9195e0c66a2f"
github_api_url="https://api.github.com"

owner = 'boyd-mask'
repo = 'git-jenkins-test'

git_current_commit_id = "7262d522d049073e4637dcee6404c5491f7297bd"
git_current_branch = "origin/pr/3/merge"


In [53]:
/usr/local/bin/fab -f /usr/local/jenkins/fabfile.py 
GithubCreateComment:
oauth_token=${OAUTH_TOKEN},
github_api_url=${GITHUB_API_URL},
owner=${OWNER},
repo=${REPO},
git_current_branch=${GIT_BRANCH},
path_log=/usr/local/opt/jenkins/jobs/${UpperJobName}/builds/${UpperBuildNumber}/log

'3'